In [31]:
# remake datasets
import torchvision.transforms as transforms
import PIL.Image as Image
import torch
from inference import get_latents
from utils.model_utils import setup_model
device = torch.device('cuda')
import os
dir_name = 'makeup'
latent_path = os.path.join('/media/pc/hengda1t/hengda/datasets/latents/',dir_name)
latent_makeup_dir = os.listdir(latent_path)
print(len(latent_makeup_dir))
# latents
latent_files = []
for x in latent_makeup_dir:
    name = os.path.join(dir_name,x)
    latent_files.append(name.replace('.pt','.png'))
print(latent_files[0])
set_latents = set(latent_files)

# lms
lms_path = os.path.join('/media/pc/hengda1t/hengda/datasets/MT-Dataset/lms',dir_name)
lms_files = os.listdir(lms_path)
lms_names = []
for x in lms_files:
    name = os.path.join(dir_name,x)
    lms_names.append(name.replace('.npy','.png'))
print(lms_names[0])
set_lms = set(lms_names)

#seg
seg_path = os.path.join('/media/pc/hengda1t/hengda/datasets/MT-Dataset/segs',dir_name)
seg_files = os.listdir(seg_path)
print('seg_files:',seg_files[0])
seg_names = []
for x in seg_files:
    name = os.path.join(dir_name,x)
    seg_names.append(name)
print('seg:',seg_names[0])
set_seg = set(seg_names)


a = set_seg & set_lms & set_latents
print(len(set_latents))
print(len(set_lms))
print(set_seg<=set_lms)
print(len(a))
with open(dir_name+'.txt','w') as f :
    for x in a:
        f.write(x +'\n')

2705
makeup/012ce0437aec38b7b42918f3e4971c13.png
makeup/012ce0437aec38b7b42918f3e4971c13.png
seg_files: 012ce0437aec38b7b42918f3e4971c13.png
seg: makeup/012ce0437aec38b7b42918f3e4971c13.png
2705
2697
False
2697


In [3]:
import torch
a = torch.tensor([float('nan')])
print(a)
if a is not None:
    print(a)

tensor([nan])
tensor([nan])


In [32]:
print(5e-3)

0.005


In [ ]:

#引入EG3D

from training.triplane import TriPlaneGenerator
import dnnlib
import legacy
from PIL import Image
import numpy as np
import torch
## end main
import sys
sys.path.append('/media/pc/hengda1t/hengda/e4e_eg3d/models/')
latent = torch.randn(4,14,512)
seq = torch.randn(4,197,768)
fusion = SimpleSelfCrossTransformer(num_layers=6, style_dim=512, heads=8, num_styles=14, inject_layers=None)

logit = fusion(reference=seq, styles=latent)
print(logit.shape)

In [ ]:
#pseudo ground truth
from config.configs import get_config
from config.pgtmaker_loss import AnnealingComposePGT, ComposePGT,MakeupLoss
from config.preprocess import PreProcess
import os 
import torch
from PIL import Image
from torchvision import transforms
pgt_annealing = True
configs = get_config()
img_size = configs.DATA.IMG_SIZE

device = 'cuda:2'
margins = {'eye':configs.PGT.EYE_MARGIN,
           'lip':configs.PGT.LIP_MARGIN}
pgt_maker =AnnealingComposePGT(margins, 
                configs.PGT.SKIN_ALPHA_MILESTONES, configs.PGT.SKIN_ALPHA_VALUES,
                configs.PGT.EYE_ALPHA_MILESTONES, configs.PGT.EYE_ALPHA_VALUES,
                configs.PGT.LIP_ALPHA_MILESTONES, configs.PGT.LIP_ALPHA_VALUES
            ) 
pgt_maker.eval()
dest_datasets = './dataset/MTdatasets_test/'
with open('./dataset/MTdatasets_test/makeup_test.txt','r') as f :
    makeup_names = [name.strip() for name in f.readlines()]

with open('./dataset/MTdatasets_test/non-makeup_test.txt','r') as f :
    non_makeup_names = [name.strip() for name in f.readlines()]

preprocessor = PreProcess(configs,need_parser=False)
PATH_DATA_ROOT = './dataset/MTdatasets_test/'

def load_from_file(img_name):
    image = Image.open(os.path.join(PATH_DATA_ROOT,'images',img_name)).convert('RGB')
    mask = preprocessor.load_mask(os.path.join(PATH_DATA_ROOT,'segs',img_name))
    base_name = os.path.splitext(img_name)[0]
    lms = preprocessor.load_lms(os.path.join(PATH_DATA_ROOT,'lms',f'{base_name}.npy'))
    return preprocessor.process(image,mask,lms)
# load source and reference
sources = []
references = []
imgs_s , imgs_r = [],[]
maskes_s, maskes_r = [],[]
diffs_s, diffs_r = [],[]
lmses_s,lmses_r = [],[]
for idx  in range(0,len(makeup_names)):
    name_s = non_makeup_names[idx]
    name_r = makeup_names[idx]
    print('makeup_names[%d]:' %idx,makeup_names[idx])
    print('non_makeup_names[%d]:' %idx ,non_makeup_names[idx])
    source = load_from_file(name_s)
    reference = load_from_file(name_r)
    sources.append(source)
    references.append(reference)
    imgs_s.append(sources[idx][0]),imgs_r.append(references[idx][0])
    maskes_s.append(sources[idx][1]),maskes_r.append(references[idx][1])
    diffs_s.append(sources[idx][2]),diffs_r.append(references[idx][2])
    lmses_s.append(sources[idx][3]),lmses_r.append(references[idx][3])
    
    pgt_A = pgt_maker(imgs_s[idx].unsqueeze(0).to(device),imgs_r[idx].unsqueeze(0).to(device),
                      maskes_s[idx].unsqueeze(0).to(device),maskes_r[idx].unsqueeze(0).to(device),
                      lmses_s[idx].unsqueeze(0).to(device),lmses_r[idx].unsqueeze(0).to(device))
    # print(pgt_A)
    image = pgt_A.cpu().clone()
    print('name_%d_torch:' %idx ,+' ',image)
    image = transforms.ToPILImage()(image.squeeze(0))
    print('name_%d_image:' %idx ,+' ',image)
    #image.save('./pgt_test/test_%d.png' %idx)


In [ ]:
# print the net parameters
import torch
import clip
print(clip)
from PIL import Image
device ="cuda:2" if torch.cuda.is_available() else "cpu"
net_farl, preprocess = clip.load("ViT-B/16", device=device)
net_farl = net_farl.to(device)
farl_state=torch.load("../pretrained_models//FaRL-Base-Patch16-LAIONFace20M-ep16.pth") # you can download from https://github.com/FacePerceiver/FaRL#pre-trained-backbones

net_farl.load_state_dict(farl_state["state_dict"],strict=False)
image = preprocess(Image.open("../save_imgs_test/inversions/xfsy_0002.jpg")).unsqueeze(0).to(device)
print(net_farl.parameters())
for name, parms in net_farl.named_parameters():
    print('-->name:', name, '-->grad_requirs:',parms.requires_grad, \
    ' -->grad_value:',parms.grad)
a = net_farl.encode_image(image)
print('a:',a.shape)